In [1]:
#szükséges csomagok meghívása
import spacy
import spacy_conll
import huspacy
from spacy import displacy

In [2]:
#nagy magyar nyelvi modell betöltése
nlp = huspacy.load()
#pipeline komponens létrehozása
nlp.add_pipe("conll_formatter", last=True)

D:\Users\t05cf\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'hu_core_news_lg' (3.4.4) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


ConllFormatter(conversion_maps=None, ext_names={'conll_str': 'conll_str', 'conll': 'conll', 'conll_pd': 'conll_pd'}, include_headers=False, disable_pandas=False)

In [3]:
#a mondat felbontása tokenekké
doc = nlp("Fazekas Zoltán Magyarországon vásárolta meg jó barátját Bellát a németjuhász kutyát a Kutyafarm Kft.től.")
for token in doc:
    print(token.text)

Fazekas
Zoltán
Magyarországon
vásárolta
meg
jó
barátját
Bellát
a
németjuhász
kutyát
a
Kutyafarm
Kft.től
.


In [4]:
#a token több, mint egy String és vannak tulajdonságai. 
#a token típusú osztály olyan, mint egy String, de több attribútuma van

for token in doc:
    print(
        token.text
        + (14 - len(token.text)) * " "
        + token.text_with_ws
        + (14 - len(token.text_with_ws)) * " "
        + str(token.is_alpha)
        + (7 - len(str(token.is_alpha))) * " "
        + str(token.is_digit)
        + (7 - len(str(token.is_digit))) * " "
        + str(token.is_upper)
        + (7 - len(str(token.is_upper))) * " "
        + str(token.i)
    )

Fazekas       Fazekas       True   False  False  0
Zoltán        Zoltán        True   False  False  1
MagyarországonMagyarországon True   False  False  2
vásárolta     vásárolta     True   False  False  3
meg           meg           True   False  False  4
jó            jó            True   False  False  5
barátját      barátját      True   False  False  6
Bellát        Bellát        True   False  False  7
a             a             True   False  False  8
németjuhász   németjuhász   True   False  False  9
kutyát        kutyát        True   False  False  10
a             a             True   False  False  11
Kutyafarm     Kutyafarm     True   False  False  12
Kft.től       Kft.től       False  False  False  13
.             .             False  False  False  14


**Szabály alapú lemmatizáló** 

Véges állapotú automatákat építhetünk nyelvészeti szabályrendszer alapján, ami "végig megy" a token karakterein és így kerül meghatározásra a lemma (és a morfológiai címkék). Az SZTE-en fejlesztett magyarlánc például ezt a technikát alkalmazza. Neurális hálók esetében pedig tanítás során kreált szabályokkal osztályozunk, amiket tudunk alkalmazni a tokenünkre, hogy megkapjuk a lemmánkat. A HuSpaCy az utóbbi kategoriába sorolható, ami egy úgy nevezett edit tree lemmatizer-t tartalmaz.

A példamondat átalakítom, hogy az egyes szavak a spaCy esetei legyenek. Majd a spaCy neurális hálón alapú lemmatizálóját használom a szavak szótári alakjának meghatározásához.

In [5]:
def print_token_feautres(token):
    print(
        token.text
        + (16 - len(token.text)) * " "
        + token.lemma_
        + (16 - len(token.lemma_)) * " "
        + token.pos_
        + (7 - len(token.pos_)) * " "
        + token.tag_
        + (7 - len(token.tag_)) * " "
        + token.dep_
        + (13 - len(token.dep_)) * " "
        + token.shape_
        + (7 - len(token.shape_)) * " "
        + str(token.is_alpha)
        + (7 - len(str(token.is_alpha))) * " "
        + str(token.is_stop)
    )

In [6]:
# tokenizálás
print("Tokenek:")
print([token.text for token in doc])

# lemmatizálás
print("\n\nLemmák:")
print([token.lemma_ for token in doc])

# POS tagging
print("\n\nPOS tagek:")
for token in doc:
    print_token_feautres(token)

Tokenek:
['Fazekas', 'Zoltán', 'Magyarországon', 'vásárolta', 'meg', 'jó', 'barátját', 'Bellát', 'a', 'németjuhász', 'kutyát', 'a', 'Kutyafarm', 'Kft.től', '.']


Lemmák:
['Fazekas', 'Zoltán', 'Magyarország', 'vásárol', 'meg', 'jó', 'barát', 'Bella', 'a', 'németjuhász', 'kutya', 'a', 'Kutyafarm', 'Kft.től', '.']


POS tagek:
Fazekas         Fazekas         PROPN  PROPN  nsubj        Xxxxx  True   False
Zoltán          Zoltán          PROPN  PROPN  flat:name    Xxxxx  True   False
Magyarországon  Magyarország    PROPN  PROPN  obl          Xxxxx  True   False
vásárolta       vásárol         VERB   VERB   ROOT         xxxx   True   False
meg             meg             PART   PART   compound:preverbxxx    True   True
jó              jó              ADJ    ADJ    amod:att     xx     True   True
barátját        barát           NOUN   NOUN   obj          xxxx   True   False
Bellát          Bella           PROPN  PROPN  obj          Xxxxx  True   False
a               a               DET    D

Tudjuk, hogy a lemma keresése függ annak szófajától is, ezért nézzük meg az egyes szavaink szófaját is. Vegyük figyelembe, hogy a szavak önállóan állnak, kontextus nélkül, így a szófaji meghatározás sok esetben pontatlan lehet.

A morfológia a nyelvészet azon ága, amely a szavak belső szerkezetét vizsgálja. A magyar nyelv alapvetően egy agglutináló nyelv, ami azt jelenti, hogy a szavaknak a szótövéhez végződések tartozhatnak, illetve akár előtag is társulhat egy adott szóhoz. A magyarban ezek a toldalékok fejezik ki a mondat szavainak egymáshoz való viszonyát, ellentétben például az angollal, ahol ezeket főként különálló, funkciószavak írják le. Az angolban így a magyarhoz képest nagyon kevés toldalalék létezik és azokat le lehet kezelni pár egyszerű szabállyal. Éppen ezért lesz nehezebb a magyar nyelvre egy morfológia elemzést végrehajtani, mert egy egyszerű előtag vagy toldalalék képes megváltoztatni a szó szerkezetét, jelentését.**POS magyarázat:**

* ADJ: melléknév
*ADP: adpozíció
*ADV: határozószó
*AUX: segéd
*CCONJ: koordináló konjunkció
*DET: meghatározó
*INTJ: közbeszólás
*FŐNÉV: főnév
*NUM: szám
*RÉSZ: részecske
*PRON: névmás
*PROPN: tulajdonnév
*PUNCT: írásjelek
*SCONJ: alárendelő kötőszó
*SYM: szimbólum
*IGE: ige
*X: egyéb


**Az oszlopok jelentése:**
* az eredeti szó a szövegben
*a szó alapalakja
*POS egyszerű címke
*címke, a részletes beszédrész címkéje
*szintaktikai függőség, azaz a tokenek közti kapcsolat
*a szó alakja - nagybetű, kisbetű, számjegy
*a token alfa karakter?
*a token egy stoplista része, a nyelv leggyakoribb szava?


A morfológia a nyelvészet azon ága, amely a szavak belső szerkezetét vizsgálja. A magyar nyelv alapvetően egy agglutináló nyelv, ami azt jelenti, hogy a szavaknak a szótövéhez végződések tartozhatnak, illetve akár előtag is társulhat egy adott szóhoz. A magyarban ezek a toldalékok fejezik ki a mondat szavainak egymáshoz való viszonyát, ellentétben például az angollal, ahol ezeket főként különálló, funkciószavak írják le. Az angolban így a magyarhoz képest nagyon kevés toldalalék létezik és azokat le lehet kezelni pár egyszerű szabállyal. Éppen ezért lesz nehezebb a magyar nyelvre egy morfológia elemzést végrehajtani, mert egy egyszerű előtag vagy toldalalék képes megváltoztatni a szó szerkezetét, jelentését.

In [7]:
for word in doc:
  print('{} -> {}'.format(word.text, word.morph))

Fazekas -> Case=Nom|Number=Sing
Zoltán -> Case=Nom|Number=Sing
Magyarországon -> Case=Sup|Number=Sing
vásárolta -> Definite=Def|Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin|Voice=Act
meg -> 
jó -> Case=Nom|Degree=Pos|Number=Sing
barátját -> Case=Acc|Number=Sing|Number[psor]=Sing|Person[psor]=3
Bellát -> Case=Acc|Number=Sing
a -> Definite=Def|PronType=Art
németjuhász -> Case=Nom|Degree=Pos|Number=Sing
kutyát -> Case=Acc|Number=Sing
a -> Definite=Def|PronType=Art
Kutyafarm -> Case=Nom|Number=Sing
Kft.től -> Case=Abl|Number=Sing
. -> 


In [ ]:
#megjelenítés
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro"}
sentence_spans = list(doc.sents)
#displacy.serve(sentence_spans, style="dep")
#displacy.serve(doc, style="dep")
displacy.serve(doc, style="ent")

D:\Users\t05cf\anaconda3\lib\site-packages\spacy\displacy\__init__.py:103: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

